# Predicting House Prices

For more information regarding this dataset, visit: 
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data.

This script will contain an initial exploration of the data, followed by some necessary preprocessing, 
feature enineering and selection, and finally the models and results.

## Some general information regarding the data

..................................


In [1]:
#call libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pylab
import math
import seaborn as sns

# Set default matplot figure size
pylab.rcParams['figure.figsize'] = (6.5, 5.0)

#Turn off pandas warning for changing variables & future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [2]:
#set random seed
np.random.seed(123)

In [3]:
#Import dataset and look at what information we are dealing with
houses = pd.read_csv("C:\\Users\\Jeroen\\Desktop\\Kaggle Datasets\\House Prices\\Excel Files\\train.csv", header = 0)
houses.head(10)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


### What does it mean!?!?!?

There are clearly many values that do not make sense for now. The meaning of all values are explained in the textfile which accompanied the dataset. Let's first check the dimensions of this dataset


In [4]:
#Look at how much data we are exactly dealing with
print("The data contains", houses.shape[0],"rows and", houses.shape[1],
      "columns. That equals", houses.size,"different cells with data")

The data contains 1460 rows and 81 columns. That equals 118260 different cells with data


## Wow!
That's a lot of data. From the header it seems that some columns contain many NaN's, which seems logical considering that not every house has certain features. Lets see what columns contain how many NaN's:

In [5]:
columns_na = houses.isna().sum()
columns_na[columns_na > 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [6]:
#There are many NA's in several categories which are not really NA's. Let's adress this. 

#Alley
houses["Alley"] = houses["Alley"].fillna(value = "NoAlley")

#Basement
houses["BsmtQual"] = houses["BsmtQual"].fillna(value = "NoBsmt")
houses["BsmtQual"] = houses["BsmtQual"].fillna(value = "NoBsmt")
houses["BsmtExposure"] = houses["BsmtExposure"].fillna(value = "NoBsmt")
houses["BsmtFinType1"] = houses["BsmtFinType1"].fillna(value = "NoBsmt")
houses["BsmtFinType2"] = houses["BsmtFinType2"].fillna(value = "NoBsmt")
houses["BsmtCond"] = houses["BsmtCond"].fillna(value = "NoBsmt")

#Fireplace
houses["FireplaceQu"] = houses["FireplaceQu"].fillna(value = "NoFirePlace")

#Garage
houses["GarageType"] = houses["GarageType"].fillna(value = "NoGarage")
houses["GarageYrBlt"] = houses["GarageYrBlt"].fillna(value = "NoGarage")
houses["GarageFinish"] = houses["GarageFinish"].fillna(value = "NoGarage")
houses["GarageQual"] = houses["GarageQual"].fillna(value = "NoGarage")
houses["GarageCond"] = houses["GarageCond"].fillna(value = "NoGarage")

#Pool
houses["PoolQC"] = houses["PoolQC"].fillna(value = "NoPool")

#Fence
houses["Fence"] = houses["Fence"].fillna(value = "NoFence")

#Misc
houses["MiscFeature"] = houses["MiscFeature"].fillna(value = "NoMisc")

In [7]:
columns_na = houses.isna().sum()
columns_na[columns_na > 0]

LotFrontage    259
MasVnrType       8
MasVnrArea       8
Electrical       1
dtype: int64